In [1]:
import tensorflow as tf

In [2]:
import math
import os

# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation
# from keras.optimizers import SGD
# from keras.utils.np_utils import to_categorical

import numpy as np
import h5py
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

In [3]:
# import pandas as pd

In [4]:
# import pickle

In [5]:
# data_path = 'data/'

In [7]:
# diagnosed_data = pd.read_csv(data_path + '22_COMB_diag_hotData.csv', low_memory=False)
# diagnosed_col = pd.read_csv(data_path + '22_COMB_diag_col.csv', low_memory=False)

# if 'Unnamed: 0' in list(diagnosed_col):
# 	diagnosed_col = diagnosed_col.drop('Unnamed: 0',axis=1,errors='ignore')

# assert (diagnosed_data.shape[0] == diagnosed_col.shape[0])
# split_index = int(diagnosed_data.shape[0] * 0.85)

# print('   ')
# print('Splitting train and test data in ratio 85:15')
# train_data = np.array(diagnosed_data.astype(float))[:split_index]
# train_label = np.array(diagnosed_col.astype(float))[:split_index][:,0]

# test_data = np.array(diagnosed_data.astype(float))[split_index:]
# test_label = np.array(diagnosed_col.astype(float))[split_index:][:,0]

   
Splitting train and test data in ratio 85:15


In [9]:
# pred_list = [1.0,2.0,3.0,7.0,9.0,19.0,21.0,99.0]
# new_list = [1,2,3,4,5,6,7,8]

In [10]:
# # Replace Label No 99 by 32
# # Label No 99 causes 'to_categorical' to make 100 one-hot values
# # Replacing it by 33 leads to only 33 values
# def replace_99_labes(label_data):
#     for i in range(len(label_data)):
#         e = label_data[i]
#         if e in pred_list:
#             ind = pred_list.index(label_data[i])
#             new_element = new_list[ind]
#             label_data[i] = new_element
#         else:
#             label_data[i] = 0
# # 		if label_data[i] == 99.0 :
# # 			label_data[i] = 32.0
#     return label_data

In [11]:
# to_keep_value = 1.0

# def binary_labels(label_data):
#     for i in range(len(label_data)):
#         if label_data[i] != 1.0 :
#             label_data[i] = 0.0
# # 		if label_data[i] == 99.0 :
# # 			label_data[i] = 32.0
#     return label_data

In [12]:
# train_rep = replace_99_labes(train_label)
# test_rep = replace_99_labes(test_label)

# # train_rep = binary_labels(train_label)
# # test_rep = binary_labels(test_label)

In [16]:
# from sklearn.preprocessing import OneHotEncoder

# def onehot_labels(labels):
#     print(labels.shape)
#     if len(labels.shape) < 2:
#         labels = labels.reshape(-1,1)
#     enc = OneHotEncoder()
#     return (enc.fit_transform(labels).toarray())

# # (134330,1)  -->  (134330, 9)    

In [17]:
# test_label_hot = onehot_labels(test_rep)
# train_label_hot = onehot_labels(train_rep)

# print(train_label_hot.shape, test_label_hot.shape)

(23706,)
(134330,)
((134330, 9), (23706, 9))


In [3]:
data_path = '/home/physics/btech/ph1140797/AHS-ML-Project/data/'

train_data = np.load(data_path + '22_COMB_diag_train_data.npy')
test_data = np.load(data_path + '22_COMB_diag_test_data.npy')
train_label = np.load(data_path + '22_COMB_diag_train_label.npy')
test_label = np.load(data_path + '22_COMB_diag_test_label.npy')

In [4]:
print(train_data.shape, train_label.shape)
print(test_data.shape, test_label.shape)

((118527, 302), (118527, 7))
((39509, 302), (39509, 7))


In [5]:
def acc(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 2).T == labels) / predictions.shape[1] / predictions.shape[0])

In [7]:
LOGDIR = '/home/physics/btech/ph1140797/AHS-ML-Project/tf_logs'

def hidden_layer(input, size_in, size_out, name='hidden'):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
        act = tf.nn.relu(tf.matmul(input, w) + b)
        
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

graph_ahs = tf.Graph()

with graph_ahs.as_default():
    SIZE = train_data.shape[1]
    HOT = train_label.shape[1]

    X = tf.placeholder(tf.float32, [None, SIZE], name='X')
    Y_ = tf.placeholder(tf.float32, [None, HOT], name='Labels')
    
    # Learning Rate - alpha
    alpha = tf.placeholder(tf.float32)
    # Dropout Probablity
    pkeep = tf.placeholder(tf.float32)
    
    K = 100    # Neurons in 1st hidden layer
    L = 75     # Neurons in 2nd hidden layer
    M = 50     # Neurons in 3rd hidden layer
    
    Y1 = hidden_layer(X, SIZE, K, name='hidden-1')
    Y2 = hidden_layer(Y1, K, L, name='hidden-2')
    Y3 = hidden_layer(Y2, L, M, name='hidden-3')
    Y_drop = tf.nn.dropout(Y3, pkeep, name='dropout')
    YF = hidden_layer(Y_drop, M, HOT, name='output')
    
    Y_f = tf.nn.softmax(YF)
    
    print('Y1 : ',Y1.get_shape())
    print('Y2 : ',Y2.get_shape())
    print('Y3 : ',Y3.get_shape())
    print('YF : ',YF.get_shape())
        
    with tf.name_scope('Cross_Entropy'):
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y_f, labels=Y_)) 
        tf.summary.scalar('Cross_Entropy', cross_entropy)
    
    with tf.name_scope('Train'):
        train_step = tf.train.AdamOptimizer(alpha).minimize(cross_entropy)
    
    with tf.name_scope('Accuracy'):
        correct_prediction = tf.equal(tf.argmax(Y_f,1), tf.argmax(Y_,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('Accuracy', accuracy)
    
    summary = tf.summary.merge_all()
    model_saver = tf.train.Saver()

('Y1 : ', TensorShape([Dimension(None), Dimension(100)]))
('Y2 : ', TensorShape([Dimension(None), Dimension(75)]))
('Y3 : ', TensorShape([Dimension(None), Dimension(50)]))
('YF : ', TensorShape([Dimension(None), Dimension(7)]))


In [9]:
train_data = train_data
label_data = train_label
print('train : ', train_data.shape, '  label : ', label_data.shape)

batch_size = 100
num_batches = int(label_data.shape[0] / batch_size)
num_epochs = 500

with tf.Session(graph=graph_ahs) as session:
    writer = tf.summary.FileWriter(LOGDIR, graph_ahs)    
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for epoch in range(num_epochs - 1):
        print('Epoch : ', epoch+1)
#         res_epoch = {}
#         t_data = svhn_test_box_dataset[epoch*test_batch:(epoch+1)*test_batch]
#         t_label = svhn_test_box_labels[epoch*test_batch:(epoch+1)*test_batch]
        
        for step in range(num_batches - 1):
            max_learning_rate = 0.001
            min_learning_rate = 0.00001

            decay_speed = 5000.0
            learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-step/decay_speed)

            batch_data = train_data[step*batch_size:(step + 1)*batch_size]
            batch_labels = label_data[step*batch_size:(step + 1)*batch_size]
            feed_dict = {X : batch_data, Y_ : batch_labels, pkeep : 0.50, alpha : learning_rate}
            
            session.run([train_step, cross_entropy], feed_dict=feed_dict)
            
            if (step % 500 == 0):
                _, l, accc = session.run([train_step, cross_entropy, accuracy], feed_dict=feed_dict)
                print('Loss at step %d: %f' % (step, l))
                print('Minibatch accuracy: %.3f%%' % (accc*100))
                # print('Minibatch error: %.3f%%' % (error*100))
                print('    ')
        print('-----------------')
        
        print ("training error is ", session.run(accuracy, feed_dict = {X : train_data, Y_ : label_data, pkeep : 0.50, alpha : learning_rate}))
            
    print('Training Complete on MNIST Data')
    
    save_path = model_saver.save(session, 'models/tensorflow/vanilla/500_itr_plain.ckpt')
    print("Model saved in file: %s" % save_path)

('train : ', (118527, 302), '  label : ', (118527, 7))
Initialized
('Epoch : ', 1)
Loss at step 0: 1.990269
Minibatch accuracy: 3.000%
    
Loss at step 500: 1.834825
Minibatch accuracy: 33.000%
    
Loss at step 1000: 1.805436
Minibatch accuracy: 36.000%
    
-----------------
('training error is ', 0.36785728)
('Epoch : ', 2)
Loss at step 0: 1.695419
Minibatch accuracy: 47.000%
    
Loss at step 500: 1.834920
Minibatch accuracy: 33.000%
    
Loss at step 1000: 1.792432
Minibatch accuracy: 37.000%
    
-----------------
('training error is ', 0.37271693)
('Epoch : ', 3)
Loss at step 0: 1.695625
Minibatch accuracy: 47.000%
    
Loss at step 500: 1.835335
Minibatch accuracy: 33.000%
    
Loss at step 1000: 1.805822
Minibatch accuracy: 36.000%
    
-----------------
('training error is ', 0.37398249)
('Epoch : ', 4)
Loss at step 0: 1.694857
Minibatch accuracy: 47.000%
    
Loss at step 500: 1.835362
Minibatch accuracy: 33.000%
    
Loss at step 1000: 1.806083
Minibatch accuracy: 36.000%


ValueError: Parent directory of models/tensorflow/vanilla/500_itr_plain.ckpt doesn't exist, can't save.

In [11]:
LOGDIR = '/home/physics/btech/ph1140797/AHS-ML-Project/tf_logs'

def hidden_layer(input, size_in, size_out, tst, name='hidden', drop=True, iter=1):
    with tf.name_scope(name):
        w = tf.get_variable(name+"W", [size_in, size_out], initializer = tf.contrib.layers.xavier_initializer())
        # w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
        y = tf.matmul(input, w)
        Y_bn, update_ema = batchnorm(y, tst, iter, b)
        act = tf.nn.relu(Y_bn)
        
        if drop:
            act = tf.nn.dropout(act, 0.50)
        
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act
    
def batchnorm(Ylogits, is_test, iteration, offset):
    exp_moving_avg = tf.train.ExponentialMovingAverage(0.999, iteration) # adding the iteration prevents from averaging across non-existing iterations
    bnepsilon = 1e-5
    
    mean, variance = tf.nn.moments(Ylogits, [0])
    update_moving_everages = exp_moving_avg.apply([mean, variance])
    m = tf.cond(is_test, lambda: exp_moving_avg.average(mean), lambda: mean)
    v = tf.cond(is_test, lambda: exp_moving_avg.average(variance), lambda: variance)
    Ybn = tf.nn.batch_normalization(Ylogits, m, v, offset, None, bnepsilon)
    return Ybn, update_moving_everages

graph_ahs = tf.Graph()

with graph_ahs.as_default():
    SIZE = train_data.shape[1]
    HOT = train_label.shape[1]

    X = tf.placeholder(tf.float32, [None, SIZE], name='X')
    Y_ = tf.placeholder(tf.float32, [None, HOT], name='Labels')
    
    # Learning Rate - alpha
    alpha = tf.placeholder(tf.float32)
    # Dropout Probablity
    pkeep = tf.placeholder(tf.float32)
    
    iter = tf.placeholder(tf.int32)
    tst = tf.placeholder(tf.bool)
    
    W_in = tf.get_variable("W", [SIZE, 200], initializer = tf.contrib.layers.xavier_initializer())
    b_in = tf.Variable(tf.constant(0.1, shape=[200]), name="B")
    Y_in = tf.nn.relu(tf.matmul(X, W_in) + b_in)
    
    Y1 = hidden_layer(Y_in, 200, 300, tst, name='hidden-1', iter=iter)
    Y2 = hidden_layer(Y1, 300, 200, tst, name='hidden-2', iter=iter)
    Y3 = hidden_layer(Y2, 200, 300, tst, name='hidden-3', iter=iter)
    Y4 = hidden_layer(Y3, 300, 200, tst, name='hidden-4', iter=iter)
    Y5 = hidden_layer(Y4, 200, 300, tst, name='hidden-5', iter=iter)
    Y6 = hidden_layer(Y5, 300, 200, tst, name='hidden-6', iter=iter)
    Y7 = hidden_layer(Y6, 200, 300, tst, name='hidden-7', iter=iter)
    Y8 = hidden_layer(Y7, 300, 200, tst, name='hidden-8', iter=iter)
    Y9 = hidden_layer(Y8, 200, 300, tst, name='hidden-9', iter=iter)
    Y10 = hidden_layer(Y9, 300, 200, tst, name='hidden-10', iter=iter)
    Y11 = hidden_layer(Y10, 200, 300, tst, name='hidden-11', iter=iter)
    Y12 = hidden_layer(Y11, 300, 200, tst, name='hidden-12', iter=iter)
    Y13 = hidden_layer(Y12, 200, 300, tst, name='hidden-13', iter=iter)
    Y14 = hidden_layer(Y13, 300, 200, tst, name='hidden-14', iter=iter)
    Y15 = hidden_layer(Y14, 200, 300, tst, name='hidden-15', iter=iter)
    Y16 = hidden_layer(Y15, 300, 200, tst, name='hidden-16', iter=iter)
    Y17 = hidden_layer(Y16, 200, 300, tst, name='hidden-17', iter=iter)
    Y18 = hidden_layer(Y17, 300, 200, tst, name='hidden-18', iter=iter)
    Y19 = hidden_layer(Y18, 200, 300, tst, name='hidden-19', iter=iter)
    Y20 = hidden_layer(Y19, 300, 200, tst, name='hidden-20', iter=iter)
    
    YF = hidden_layer(Y20, 200, HOT, tst, name='output', drop=False)
    
    Y_f = tf.nn.softmax(YF)
    
    print('Y1 : ',Y1.get_shape())
    print('Y2 : ',Y2.get_shape())
    print('Y3 : ',Y3.get_shape())
    print('YF : ',YF.get_shape())
        
    with tf.name_scope('Cross_Entropy'):
        cross_entropy =  tf.reduce_mean(-tf.reduce_sum(Y_ * tf.log(tf.clip_by_value(Y_f, 1e-8, 1.0)), reduction_indices=[1]))
        # cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y_f, labels=Y_)) 
        tf.summary.scalar('Cross_Entropy', cross_entropy)
    
    with tf.name_scope('Train'):
        train_step = tf.train.AdamOptimizer(.00006).minimize(cross_entropy)
    
    with tf.name_scope('Accuracy'):
        correct_prediction = tf.equal(tf.argmax(Y_f,1), tf.argmax(Y_,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('Accuracy', accuracy)
    
    mistakes = tf.not_equal(tf.argmax(Y_, 1), tf.argmax(Y_f, 1))
    error = tf.reduce_mean(tf.cast(mistakes, tf.float32))
    
    summary = tf.summary.merge_all()
    model_saver = tf.train.Saver()

('Y1 : ', TensorShape([Dimension(None), Dimension(300)]))
('Y2 : ', TensorShape([Dimension(None), Dimension(200)]))
('Y3 : ', TensorShape([Dimension(None), Dimension(300)]))
('YF : ', TensorShape([Dimension(None), Dimension(7)]))


In [ ]:
train_data = train_data
label_data = train_label
print('train : ', train_data.shape, '  label : ', label_data.shape)

batch_size = 100
num_batches = int(label_data.shape[0] / batch_size)
num_epochs = 500

with tf.Session(graph=graph_ahs) as session:
    writer = tf.summary.FileWriter(LOGDIR, graph_ahs)    
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for epoch in range(num_epochs - 1):
        print('Epoch : ', epoch+1)
#         res_epoch = {}
#         t_data = svhn_test_box_dataset[epoch*test_batch:(epoch+1)*test_batch]
#         t_label = svhn_test_box_labels[epoch*test_batch:(epoch+1)*test_batch]
        
        for step in range(num_batches - 1):
            max_learning_rate = 0.001
            min_learning_rate = 0.00001

            decay_speed = 5000.0
            learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-step/decay_speed)

            batch_data = train_data[step*batch_size:(step + 1)*batch_size]
            batch_labels = label_data[step*batch_size:(step + 1)*batch_size]
            feed_dict = {X : batch_data, Y_ : batch_labels, pkeep : 0.50, alpha : learning_rate, tst : False, iter : step}
            
            session.run([train_step, cross_entropy], feed_dict=feed_dict)
            
            if (step % 500 == 0):
                _, l, accc = session.run([train_step, cross_entropy, accuracy], feed_dict=feed_dict)
                print('Loss at step %d: %f' % (step, l))
                print('Minibatch accuracy: %.3f%%' % (accc*100))
                # print('Minibatch error: %.3f%%' % (error*100))
                print('    ')
        print('-----------------')
        
        print ("training error is ", session.run(accuracy, feed_dict = {X : train_data, Y_ : label_data, pkeep : 0.50, alpha : learning_rate, tst : True, iter : epoch}))
            
    print('Training Complete on MNIST Data')
    
    save_path = model_saver.save(session, 'models/1_label/label_1.ckpt')
    print("Model saved in file: %s" % save_path)

('train : ', (118527, 302), '  label : ', (118527, 7))
Initialized
('Epoch : ', 1)
Loss at step 0: 2.124584
Minibatch accuracy: 16.000%
    
Loss at step 500: 2.145754
Minibatch accuracy: 11.000%
    
Loss at step 1000: 2.216960
Minibatch accuracy: 11.000%
    
-----------------
('training error is ', 0.23484944)
('Epoch : ', 2)
Loss at step 0: 2.095533
Minibatch accuracy: 16.000%
    
Loss at step 500: 2.084682
Minibatch accuracy: 14.000%
    
Loss at step 1000: 1.917107
Minibatch accuracy: 21.000%
    
-----------------
('training error is ', 0.36722445)
('Epoch : ', 3)
Loss at step 0: 1.902304
Minibatch accuracy: 28.000%
    
Loss at step 500: 1.906986
Minibatch accuracy: 29.000%
    
Loss at step 1000: 1.906731
Minibatch accuracy: 28.000%
    
-----------------
('training error is ', 0.36782357)
('Epoch : ', 4)
Loss at step 0: 1.870644
Minibatch accuracy: 38.000%
    
Loss at step 500: 1.870259
Minibatch accuracy: 28.000%
    
Loss at step 1000: 1.881019
Minibatch accuracy: 38.000%

In [ ]:
train_data = train_data
label_data = train_label_hot
print('train : ', train_data.shape, '  label : ', label_data.shape)

batch_size = 100
num_batches = int(label_data.shape[0] / batch_size)
num_epochs = 200

with tf.Session(graph=graph_ahs) as session:
    writer = tf.summary.FileWriter(LOGDIR, graph_ahs)    
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for epoch in range(num_epochs - 1):
        print('Epoch : ', epoch+1)
#         res_epoch = {}
#         t_data = svhn_test_box_dataset[epoch*test_batch:(epoch+1)*test_batch]
#         t_label = svhn_test_box_labels[epoch*test_batch:(epoch+1)*test_batch]
        
        for step in range(num_batches - 1):
            max_learning_rate = 0.001
            min_learning_rate = 0.00001

            decay_speed = 5000.0
            learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-step/decay_speed)

            batch_data = train_data[step*batch_size:(step + 1)*batch_size]
            batch_labels = label_data[step*batch_size:(step + 1)*batch_size]
            feed_dict = {X : batch_data, Y_ : batch_labels, pkeep : 0.50, alpha : learning_rate}
            
            session.run([train_step, cross_entropy], feed_dict=feed_dict)
            
            if (step % 500 == 0):
                _, l, accc = session.run([train_step, cross_entropy, accuracy], feed_dict=feed_dict)
                print('Loss at step %d: %f' % (step, l))
                print('Minibatch accuracy: %.3f%%' % (accc*100))
                # print('Minibatch error: %.3f%%' % (error*100))
                print('    ')
        print('-----------------')
        
        print ("training error is ", session.run(accuracy, feed_dict = {X : train_data, Y_ : label_data, pkeep : 0.50, alpha : learning_rate}))
            
    print('Training Complete on MNIST Data')
    
    save_path = model_saver.save(session, 'models/1_label/label_1.ckpt')
    print("Model saved in file: %s" % save_path)

In [ ]:
hidden_dict = {}

def hidden_layer(input, size_in, size_out, name='hidden'):
    with tf.name_scope(name):
#         print(input.get_shape())
        w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
#         print(w.get_shape())
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
#         print(b.get_shape())
        act = tf.nn.relu(tf.matmul(input, w) + b)
        
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

def test_layer_performance(no_of_hidden_layers) :
    graph_a = tf.Graph()
    with graph_a.as_default():
#         train_data = train_data
        label_data = train_label_hot

        SIZE = train_data.shape[1]
        HOT = train_label_hot.shape[1]

        X = tf.placeholder(tf.float32, [None, SIZE], name='X')
        Y_ = tf.placeholder(tf.float32, [None, HOT], name='Labels')

        # Learning Rate - alpha
        alpha = tf.placeholder(tf.float32)
        # Dropout Probablity
        pkeep = tf.placeholder(tf.float32)

        K = 100    # Neurons in 1st hidden layer
        L = 75     # Neurons in 2nd hidden layer
        M = 50     # Neurons in 3rd hidden layer

        Y1 = hidden_layer(X, SIZE, K, name='hidden-1')
        Y2 = hidden_layer(Y1, K, L, name='hidden-2')
        Y3 = hidden_layer(Y2, L, M, name='hidden-3')
        
        hidden_list = list()
        
        for i in range(no_of_hidden_layers):
            name = 'hidden-deep' + str(i)
            if i == 0:
                hidden_list.append(hidden_layer(Y3, M, M, name=name))
            else :
                y = hidden_list[i-1]
                hidden_list.append(hidden_layer(y, M, M, name=name))

#         for h in hidden_list:
#             print(h.get_shape())
#             print(h.name)
#         print(hidden_list)
                
#         print(len(hidden_list))
        Y_drop = tf.nn.dropout(hidden_list[-1], pkeep, name='dropout')
        YF = hidden_layer(Y_drop, M, HOT, name='output')
        Y_f = tf.nn.softmax(YF)

        with tf.name_scope('Cross_Entropy'):
            cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y_f, labels=Y_)) 
            tf.summary.scalar('Cross_Entropy', cross_entropy)

        with tf.name_scope('Train'):
            train_step = tf.train.AdamOptimizer(alpha).minimize(cross_entropy)

        with tf.name_scope('Accuracy'):
            correct_prediction = tf.equal(tf.argmax(Y_f,1), tf.argmax(Y_,1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            tf.summary.scalar('Accuracy', accuracy)

        summary = tf.summary.merge_all()
        model_saver = tf.train.Saver()
        
        
    #### TRAINING START ####
    batch_size = 512
    test_batch = 512
    num_steps = int(label_data.shape[0] / batch_size)
    num_epochs = int(test_label_hot.shape[0]/test_batch)

    with tf.Session(graph=graph_a) as session:
        writer = tf.summary.FileWriter(LOGDIR, graph_a)
        tf.global_variables_initializer().run()
        print('Initialized')

        test_acc = list()
        train_acc = list()
        for epoch in range(num_epochs - 1):
            t_data = test_data[epoch*test_batch:(epoch+1)*test_batch]
            t_label = test_label_hot[epoch*test_batch:(epoch+1)*test_batch]

            for step in range(num_steps - 1):
                max_learning_rate = 0.0005
                min_learning_rate = 0.0001

                decay_speed = 5000.0
                learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-step/decay_speed)

                batch_data = train_data[step*batch_size:(step + 1)*batch_size,:]
                batch_labels = label_data[step*batch_size:(step + 1)*batch_size]
                feed_dict = {X : batch_data, Y_ : batch_labels, pkeep : 0.75, alpha : learning_rate}

                _, l, accc = session.run([train_step, cross_entropy, accuracy], feed_dict=feed_dict)
                train_acc.append(accc)

            # Test Accuracy
            feed_dict = {X : t_data, Y_ : t_label, pkeep : 1.00, alpha : 0.00}
            _, l, accc = session.run([train_step, cross_entropy, accuracy], feed_dict=feed_dict)
            test_acc.append(accc)
        
        test_accuracy = np.mean(test_acc)
        print('Train Accuract : ', np.mean(train_acc))
        print('Test Accuracy : ', test_accuracy)
        hidden_dict[no_of_hidden_layers] = test_accuracy
        
        save_path = model_saver.save(session, 'models/hyp_tune/' + str(no_of_hidden_layers) + '.ckpt')
        print("Model saved in file: %s" % save_path)
        print('-----------------------')

In [ ]:
for i in range(1,10):
    test_layer_performance(i)
    
print hidden_dict

In [ ]:
file = 'models/hidden_dp_75.pickle'

with open(file, 'wb') as handle:
    pickle.dump(hidden_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)